In [2]:
!pip install psycopg2-binary pandas tqdm

     |████████████████████████████████| 3.0 MB 1.8 MB/s eta 0:00:01


In [1]:
targets = [
    "Cytochrome P450 19A1",
    "Muscarinic acetylcholine receptor M2",
    "Muscarinic acetylcholine receptors; M2 & M3",
    "Dopamine D3 receptor",
    "Dopamine D4 receptor",
    "Neprilysin",
    "Plasminogen",
    "Muscarinic acetylcholine receptor M1",
    "Angiotensin II receptor",
    "Serotonin 1a (5-HT1a) receptor",
    "Matrix metalloproteinase 3",
    "Matrix metalloproteinase-1",
    "Matrix metalloproteinase 9",
    "Matrix metalloproteinase-2",
    "ADAM17",
    "Peroxisome proliferator-activated receptor gamma",
    "Receptor protein-tyrosine kinase erbB-2",
    "Tyrosine-protein kinase LCK",
    "Angiotensin II type 2 (AT-2) receptor",
    "Serum albumin",
    "Adenosine A2a receptor",
    "Intercellular adhesion molecule-1",
    "Androgen Receptor",
    "Estrogen receptor",
    "Purinergic receptor P2Y2",
    "Excitatory amino acid transporter 1",
    "Excitatory amino acid transporter 2",
    "Excitatory amino acid transporter 3",
    "Caspase-1",
    "Caspase-3",
    "Caspase-6",
    "Caspase-8",
    "Renin",
    "Sigma opioid receptor",
    "Nitric oxide synthase, inducible",
    "Nitric-oxide synthase, endothelial",
    "Nitric-oxide synthase, brain",
    "Nitric oxide synthase (inducible and endothelial)",
    "Nitric oxide sythases; iNOS & nNOS",
    "P-selectin",
    "Complement C1r",
    "Type-1 angiotensin II receptor",
    "Serotonin 2c (5-HT2c) receptor",
    "Serotonin 2a (5-HT2a) receptor",
    "Monoamine oxidase A",
    "Butyrylcholinesterase",
    "Acetylcholinesterase",
    "Lipoprotein lipase",
    "Dopamine D1 receptor",
    "Cathepsin D",
    "Cathepsin G",
    "Cyclooxygenase-2",
    "Cyclooxygenase-1",
    "Cyclooxygenase",
    "Alpha-2a adrenergic receptor",
    "Dopamine transporter",
    "Serotonin 6 (5-HT6) receptor",
    "Calpain 1",
    "Glutamate receptor ionotropic, AMPA 1",
    "Glutamate receptor ionotropic, AMPA 2",
    "Glutamate receptor ionotropic, AMPA 3",
    "Cannabinoid CB2 receptor",
    "Cathepsin B",
    "Beta-secretase 1",
    "Adrenergic receptor beta",
    "Dopamine receptors; D3 & D4",
    "11-beta-hydroxysteroid dehydrogenase 1",
    "Monoamine oxidase B",
    "Tissue-type plasminogen activator",
    "Cytochrome P450 2D6",
    "HMG-CoA reductase",
    "TNF-beta",
    "Beta-2 adrenergic receptor",
    "Phospholipase A2 group IIA",
    "Serine/threonine-protein kinase RAF",
    "Gamma-secretase",
    "Estrogen receptor alpha",
    "Estrogen receptor beta",
    "Melatonin receptor 1A",
    "Melatonin receptor",
    "Arachidonate 5-lipoxygenase",
    "Muscarinic acetylcholine receptor M2 and M4",
    "Muscarinic acetylcholine receptor M2 and M5",
    "Beta-glucocerebrosidase",
    "Serum paraoxonase/arylesterase 1",
    "Angiotensin-converting enzyme",
    "Dopamine receptor",
    "GABA-B receptor",
    "Muscarinic acetylcholine receptor",
    "Cytochrome P450 17A1",
    "FK506 binding protein 12",
    "Cyclin-dependent kinase 1",
    "Beta amyloid A4 protein",
    "Farnesyl diphosphate synthase",
    "Corticotropin releasing factor receptor 1",
    "Cyclin-dependent kinase 5/CDK5 activator 1",
    "Glycogen synthase kinase-3",
    "Neuronal acetylcholine receptor; alpha4/beta2",
    "Neuronal acetylcholine receptor protein alpha-7 subunit",
    "Gamma-secretase subunit PEN-2",
    "Erythropoietin receptor",
    "Purine nucleoside phosphorylase",
    "C-C chemokine receptor type 5",
    "C-C chemokine receptor type 3",
    "Urokinase-type plasminogen activator",
    "C5a anaphylatoxin chemotactic receptor",
    "Macrophage migration inhibitory factor",
    "Dual specificity mitogen-activated protein kinase kinase 1",
    "Tyrosine-protein kinase FYN",
    "Serotonin transporter",
    "Acyl coenzyme A:cholesterol acyltransferase 1",
    "Peroxisome proliferator-activated receptor alpha",
    "Peroxisome proliferator-activated receptor delta",
    "P-glycoprotein 1",
    "Tyrosine-protein kinase ABL",
    "Neuronal acetylcholine receptor; alpha3/beta4",
    "Glycogen synthase kinase-3 beta",
    "Angiotensin-converting enzyme",
    "Voltage-gated L-type calcium channel alpha-1C subunit",
    "Glycogen synthase kinase-3 alpha",
    "Monoamine oxidase",
    "Beta-secretase (BACE)",
    "Plasminogen activator inhibitor-1",
    "Matrix metalloproteinase (2 and 3)",
    "Transthyretin",
    "Monoamine transporters; serotonin & dopamine",
    "Matrix metalloproteinase 12",
    "Mitogen-activated protein kinase; ERK1/ERK2",
    "Ribosomal protein S6 kinase (P70S6K)",
    "Serine/threonine-protein kinase AKT",
    "Dual-specificity tyrosine-phosphorylation regulated kinase 1A",
    "MAP kinase ERK1",
    "MAP kinase ERK2",
    "Tyrosine-protein kinase ABL",
    "PPAR alpha/gamma",
    "Insulin-like growth factor binding protein 3",
    "TNF-alpha",
    "Receptor-type tyrosine-protein phosphatase alpha",
    "Glutamate receptor ionotropic AMPA",
    "Prolyl endopeptidase",
    "Insulin-like growth factor binding protein 2",
    "Insulin-like growth factor binding protein 6",
    "Ceramide glucosyltransferase",
    "Insulin-like growth factor I receptor",
    "Insulin receptor",
    "Dual specificity mitogen-activated protein kinase kinase; MEK1/2",
    "Glyceraldehyde-3-phosphate dehydrogenase liver",
    "Adenosine receptors; A2a & A3",
    "Caspase-9",
    "Vitamin D receptor",
    "Vesicular acetylcholine transporter",
    "Nerve growth factor receptor Trk-A",
    "Dual specificity mitogen-activated protein kinase kinase 2",
    "Subtilisin/kexin type 9",
    "GAR transformylase",
    "Anandamide amidohydrolase",
    "Quinone reductase 1",
    "Muscarinic acetylcholine receptor M1 and M3",
    "Thioredoxin reductase",
    "Caspase-7",
    "Phenylethanolamine N-methyltransferase",
    "Serotransferrin",
    "CaM kinase II",
    "Vascular endothelial growth factor A",
    "Glutamine synthetase",
    "Arachidonate 15-lipoxygenase",
    "Complement factor H",
    "Tumor necrosis factor receptor R1",
    "G-protein coupled receptor kinase 2",
    "Arachidonate 12-lipoxygenase",
    "Phosphodiesterase 4B",
    "Beta-2 adrenergic receptor and beta-3 adrenergic receptor",
    "RXR alpha/PPAR gamma",
    "Serine/threonine-protein kinase RAF and Dual specificity mitogen-activated protein kinase kinase 1 (Raf/MEK)",
    "Glutathione S-transferase Pi",
    "Cyclin-dependent kinase 5",
    "Bcr/Abl fusion protein",
    "Caspase-2",
    "Caspase-4",
    "Interleukin-8 receptor B",
    "Neuronal acetylcholine receptor; alpha2/beta4",
    "Superoxide dismutase",
    "Lysosomal acid lipase/cholesteryl ester hydrolase",
    "Neuronal acetylcholine receptor subunit alpha-3",
    "Phosphatidylcholine-retinol O-acyltransferase",
    "Cannabinoid receptor",
    "Nitric-oxide synthase (endothelial and brain)",
    "Neuronal acetylcholine receptor; alpha3/beta2",
    "Beta secretase 2",
    "Glutamate (NMDA) receptor subunit zeta 1",
    "Cystic fibrosis transmembrane conductance regulator",
    "Inositol 1,4,5-trisphosphate receptor type 1",
    "Glutamate [NMDA] receptor subunit epsilon 1",
    "Glutamate [NMDA] receptor subunit epsilon 2",
    "Poly [ADP-ribose] polymerase-1",
    "DNA polymerase beta",
    "Choline acetylase",
    "Catechol O-methyltransferase",
    "Cathepsin (B and K)",
    "C-X-C chemokine receptor type 3",
    "Glutamate NMDA receptor; GRIN1/GRIN2A",
    "Glutamate NMDA receptor; GRIN1/GRIN2B",
    "Serine/threonine-protein kinase WEE1",
    "Aryl hydrocarbon receptor",
    "Collagenase",
    "Aldehyde dehydrogenase",
    "Receptor protein-tyrosine kinase erbB-4",
    "PI3-kinase p110-alpha/p85-alpha",
    "Cytochrome P450 1A1",
    "Proenkephalin B",
    "5-lipoxygenase/FLAP",
    "LDL receptor",
    "Interleukin-8",
    "Myeloperoxidase",
    "cAMP-dependent protein kinase alpha-catalytic subunit",
    "Prion protein",
    "Low affinity neurotrophin receptor p75NTR",
    "Cerebroside-sulfatase",
    "Quinone reductase 2",
    "Suv3 helicase",
    "Ribosomal protein S6 kinase 1",
    "Serum amyloid P-component",
    "HLA class I histocompatibility antigen A-3",
    "Acidic fibroblast growth factor",
    "HLA class II histocompatibility antigen DRB1-1",
    "Peptidyl-prolyl cis-trans isomerase NIMA-interacting 1",
    "Insulin-like growth factor II receptor",
    "PPAR delta/gamma",
    "Neuronal acetylcholine recetpor; alpha2/beta2",
    "Tumour suppressor p53/oncoprotein Mdm2",
    "Prostaglandin E synthase 2",
    "Casein kinase I delta",
    "Fatty acid binding protein muscle",
    "ADAMTS4",
    "ADAM12",
    "ADAM10",
    "Breakpoint cluster region protein",
    "S-100 protein beta chain",
    "Serine/threonine-protein kinase EEF2K",
    "LXR-beta",
    "Coagulation factor XIII",
    "Kinesin-like protein 1",
    "Eukaryotic translation initation factor",
    "Guanine nucleotide-binding protein G(o), alpha subunit 1",
    "CaM kinase II alpha",
    "Ephrin type-A receptor 5",
    "CaM kinase II gamma",
    "CaM kinase I alpha",
    "Serine/threonine-protein kinase PAK 1",
    "NAD-dependent deacetylase sirtuin 1",
    "Toll-like receptor 4",
    "Phosphodiesterase 8B",
    "Synaptic vesicular amine transporter",
    "Apoptosis regulator Bcl-W",
    "Coagulation factor V",
    "ADAMTS1",
    "Neurotrophic tyrosine kinase receptor type 2",
    "Luteinizing hormone/Choriogonadotropin receptor",
    "Heme oxygenase 1",
    "Insulin",
    "Muscarinic acetylcholine receptor M1/M5 chimeric protein",
    "Catenin beta-1",
    "Phosphoenolpyruvate carboxykinase cytosolic",
    "Interferon-induced, double-stranded RNA-activated protein kinase",
    "Alpha-1-antichymotrypsin",
    "Ubiquitin carboxyl-terminal hydrolase isozyme L1",
    "Breast cancer type 1 susceptibility protein",
    "Alpha-synuclein",
    "Segment polarity protein dishevelled homolog DVL-1",
    "Nucleoside diphosphate kinase 1",
    "Nucleoside diphosphate kinase 2",
    "Cyclin-dependent kinase 5 regulatory subunit 1",
    "PI3-kinase p85-alpha subunit",
    "Ornithine aminotransferase, mitochondrial",
    "Ephrin type-A receptor 1",
    "Death-associated protein kinase 1",
    "Serine/threonine-protein kinase 11",
    "Serine/threonine-protein kinase MARK1",
    "Serine/threonine-protein kinase 24",
    "Protein tyrosine kinase 2 beta",
    "Apolipoprotein A-I",
    "Thioredoxin reductase 2",
    "NT-3 growth factor receptor",
    "Receptor tyrosine-protein kinase erbB-3",
    "Interleukin-2",
    "Presenilin 1",
    "Cellular tumor antigen p53",
    "DNA-(apurinic or apyrimidinic site) lyase",
    "Neuronal acetylcholine receptor protein alpha-4 subunit",
    "Glutathione transferase omega 1",
    "Huntingtin",
    "Macrophage scavenger receptor types I and II",
    "Cyclic AMP-responsive element-binding protein 1",
    "Disks large homolog 4",
    "Tumor necrosis factor ligand superfamily member 10",
    "Tumor necrosis factor ligand superfamily member 6",
    "Arylamine N-acetyltransferase 1",
    "Arylamine N-acetyltransferase 2",
    "Toll-like receptor 9",
    "GABA-B receptor 1",
    "Tryptophan 5-hydroxylase 1",
    "Glucagon",
    "Non-receptor tyrosine-protein kinase TNK1",
    "Histone acetyltransferase KAT5",
    "Endoplasmic reticulum aminopeptidase 1",
    "Adenylate cyclase type II",
    "Aquaporin-4",
    "Peroxiredoxin-1",
    "Acid ceramidase",
    "Heat shock 70 kDa protein 1",
    "Dynamin-2",
    "Transcription factor Sp1",
    "Creatine kinase B-type",
    "Dickkopf-related protein 1",
    "Kallikrein 8",
    "Leucine-rich repeat serine/threonine-protein kinase 2",
    "Integrin alpha M",
    "Elongation factor 2",
    "Tumor necrosis factor receptor superfamily member 5",
    "Tumor necrosis factor receptor superfamily member 1B",
    "Toll-like receptor (TLR7/TLR9)",
    "MIF/CD74 (Macrophage migration inhibitory factor and HLA-DR antigens-associated invariant chain)",
    "Eukaryotic translation initiation factor 2 subunit 1",
    "PI3-kinase p110-delta/p85-alpha",
    "C-C motif chemokine 5",
    "Transcriptional regulator ERG",
    "Prelamin-A/C",
    "Microtubule-associated protein tau",
    "Endoplasmic reticulum-associated amyloid beta-peptide-binding protein",
    "Peroxisome proliferator-activated receptor gamma/Nuclear receptor coactivator 1",
    "Peroxisome proliferator-activated receptor gamma/Nuclear receptor corepressor 2",
    "Ubiquitin carboxyl-terminal hydrolase BAP1",
    "Peroxisome proliferator-activated receptor gamma/Nuclear receptor coactivator 2",
    "Insulin-degrading enzyme",
    "Peroxisome proliferator-activated receptor gamma/Nuclear receptor coactivator 3",
    "Excitatory amino acid transporter 5",
    "Relaxin-3 receptor 1",
    "Neuronal acetylcholine recetpor; alpha6/beta2",
    "C-C motif chemokine 2",
    "Neuronal acetylcholine receptor protein beta-2 subunit",
    "Peptidyl-prolyl cis-trans isomerase D",
    "Glutathione S-transferase A4",
    "Glutathione transferase omega 2",
    "Platelet glycoprotein 4",
    "G1/S-specific cyclin D1",
    "78 kDa glucose-regulated protein",
    "Interleukin-6",
    "Somatostatin",
    "Bis(5'-adenosyl)-triphosphatase",
    "Protein polybromo-1",
    "Transforming growth factor beta-1",
    "Kallikrein 7",
    "Regulator of G-protein signaling 4",
    "Protein-arginine deiminase type-2",
    "Interleukin-1 beta",
    "Scavenger receptor class B member 1",
    "Islet amyloid polypeptide",
    "Appetite-regulating hormone",
    "Protoporphyrinogen oxidase",
    "Natural resistance-associated macrophage protein 2",
    "Proteasome subunit beta type-9",
    "DNA polymerase nu",
    "NEDD8-activating enzyme E1 regulatory subunit",
    "Neuronal acetylcholine receptors; alpha-3/beta-4, alpha-5, beta-2",
    "Kallikrein 6",
    "Phosphatidylinositol 3,4,5-trisphosphate 3-phosphatase and dual-specificity protein phosphatase PTEN",
    "Natural resistance-associated macrophage protein 1",
    "Heat shock-related 70 kDa protein 2",
    "ATP synthase subunit alpha, mitochondrial",
    "Guanine nucleotide-binding protein G(s), subunit alpha",
    "Neurogenic locus notch homolog protein 1",
    "Methionine synthase",
    "Ubiquitin carboxyl-terminal hydrolase 21",
    "Presenilin 2",
    "Advanced glycosylation end product-specific receptor",
    "Tumour suppressor protein p53/Mdm4",
    "Mammalian target of Rapamycin (mTORC1)",
    "High mobility group protein B1",
    "Alpha-ketoglutarate-dependent dioxygenase FTO",
    "TAR DNA-binding protein 43",
    "Muscarinic acetylcholine receptors; M1 & M2",
    "Neuronal acetylcholine receptor; alpha4/beta4",
    "Adenosine receptors; A1 & A2",
    "Dopamine D2 receptor and serotonin 1a receptor",
    "Cholinesterases; ACHE & BCHE",
    "Serotonin 2 receptors; 5-HT2a & 5-HT2c",
    "Serotonin receptor (2b and 2c)",
    "Neuronal acetylcholine receptor; alpha3/alpha6/beta2/beta3"
]

In [2]:
import psycopg2
import pandas as pd
from tqdm import tqdm 

def get_chembl_ids_and_fetch_data(target_names):
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname="chembl_35",
        user="chembl_maghaeef",  
        password="nasir",  
        host="localhost",
        port=5432
    )

    chembl_ids = {}

    # Step 1: Get ChEMBL IDs for target names (using SQL)
    with conn.cursor() as cursor:
        for target_name in target_names:
            cursor.execute("""
                SELECT chembl_id, target_type, organism
                FROM target_dictionary
                WHERE pref_name ILIKE %s
            """, (target_name,))
            
            results = cursor.fetchall()
            if results:
                chembl_ids[target_name] = [{
                    "target_chembl_id": row[0],
                    "target_type": row[1],
                    "organism": row[2]
                } for row in results]
            else:
                print(f"No exact match found for target: {target_name}")

    # Step 2: Fetch data using optimized SQL joins
    all_data = []
    for target_name, target_info_list in chembl_ids.items():
        for target_info in target_info_list:
            target_id = target_info["target_chembl_id"]
            print(f"Fetching data for target {target_id} ({target_name})...")
            
            try:
                # Single query to get all related data using joins
                query = """
                    SELECT 
                        a.standard_value,
                        a.standard_type AS activity_type,
                        m.chembl_id AS molecule_chembl_id,
                        m.max_phase,
                        m.molecule_type,
                        cs.canonical_smiles,
                        cs.standard_inchi,
                        cs.standard_inchi_key,
                        cp.full_molformula,
                        cp.full_mwt,
                        cp.alogp,
                        cp.aromatic_rings,
                        cp.hba,
                        cp.hbd,
                        cp.rtb,
                        cp.psa,
                        cp.qed_weighted
                    FROM activities a
                    JOIN assays ass ON a.assay_id = ass.assay_id
                    JOIN target_dictionary td ON ass.tid = td.tid
                    JOIN molecule_dictionary m ON a.molregno = m.molregno
                    JOIN compound_structures cs ON m.molregno = cs.molregno
                    JOIN compound_properties cp ON m.molregno = cp.molregno
                    WHERE td.chembl_id = %s
                    AND a.standard_value IS NOT NULL
                """
                
                with conn.cursor(name="fetch_activities") as cursor:
                    cursor.itersize = 1000  # Batch size for server-side cursor
                    cursor.execute(query, (target_id,))
                    
                    for row in tqdm(cursor, desc=f"Processing {target_id}"):
                        info = {
                            "Target Name": target_name,
                            "Target ChEMBL ID": target_id,
                            "Target Type": target_info["target_type"],
                            "Organism": target_info["organism"],
                            "Molecule ChEMBL ID": row[2],
                            "Activity Type": row[1],
                            "Activity Value": row[0],
                            "Canonical SMILES": row[5],
                            "Standard InChI": row[6],
                            "Standard InChI Key": row[7],
                            "Max Phase": row[3],
                            "Molecular Formula": row[8],
                            "Molecular Weight": row[9],
                            "AlogP": row[10],
                            "Aromatic Rings": row[11],
                            "HBA": row[12],
                            "HBD": row[13],
                            "Rotatable Bonds": row[14],
                            "Polar Surface Area (PSA)": row[15],
                            "QED Weighted": row[16]
                        }
                        all_data.append(info)

            except Exception as e:
                print(f"Error retrieving data for target {target_id}: {e}")
                conn.rollback()
            else:
                conn.commit()

    conn.close()
    return pd.DataFrame(all_data)

# Example usage
if __name__ == "__main__":
    target_names = targets
    df = get_chembl_ids_and_fetch_data(target_names)
df

No exact match found for target: Beta amyloid A4 protein
No exact match found for target: HLA class II histocompatibility antigen DRB1-1
No exact match found for target: Neuronal acetylcholine recetpor; alpha2/beta2
No exact match found for target: Excitatory amino acid transporter 5
No exact match found for target: Neuronal acetylcholine recetpor; alpha6/beta2
No exact match found for target: Somatostatin
No exact match found for target: Mammalian target of Rapamycin (mTORC1)
Fetching data for target CHEMBL1978 (Cytochrome P450 19A1)...


Processing CHEMBL1978: 6630it [00:03, 2093.89it/s]


Fetching data for target CHEMBL3859 (Cytochrome P450 19A1)...


Processing CHEMBL3859: 263it [00:01, 204.51it/s]


Fetching data for target CHEMBL211 (Muscarinic acetylcholine receptor M2)...


...

Fetching data for target CHEMBL2111466 (Serotonin receptor (2b and 2c))...


Processing CHEMBL2111466: 114it [00:00, 142.68it/s]


Fetching data for target CHEMBL2109233 (Neuronal acetylcholine receptor; alpha3/alpha6/beta2/beta3)...


Processing CHEMBL2109233: 91it [00:00, 112.77it/s]


,Target Name,Target ChEMBL ID,Target Type,Organism,Molecule ChEMBL ID,Activity Type,Activity Value,Canonical SMILES,Standard InChI,Standard InChI Key,Max Phase,Molecular Formula,Molecular Weight,AlogP,Aromatic Rings,HBA,HBD,Rotatable Bonds,Polar Surface Area (PSA),QED Weighted
0,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL63297,IC50,190.0,Fc1ccc(C(C2Cc3ccccc3O2)n2cncn2)cc1,InChI=1S/C17H14FN3O/c18-14-7-5-12(6-8-14)17(21...,RVFPOHIDBASNMF-UHFFFAOYSA-N,None,C17H14FN3O,295.32,3.01,3.0,4.0,0.0,3.0,39.94,0.75
1,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL65328,IC50,590.0,Cc1ccc(C(C2Cc3ccccc3O2)n2cncn2)cc1,InChI=1S/C18H17N3O/c1-13-6-8-14(9-7-13)18(21-1...,OZEGINYEQGNONL-UHFFFAOYSA-N,None,C18H17N3O,291.35,3.18,3.0,4.0,0.0,3.0,39.94,0.74
2,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL488,IC50,18500.0,CCC1(c2ccc(N)cc2)CCC(=O)NC1=O,InChI=1S/C13H16N2O2/c1-2-13(8-7-11(16)15-12(13...,ROBVIMPUHSLWNV-UHFFFAOYSA-N,4.0,C13H16N2O2,232.28,1.35,1.0,3.0,2.0,2.0,72.19,0.60
3,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL291646,IC50,200.0,Clc1ccc(C(C2Cc3ccccc3O2)n2cncn2)cc1,InChI=1S/C17H14ClN3O/c18-14-7-5-12(6-8-14)17(2...,UVOSSULRORMWHE-UHFFFAOYSA-N,None,C17H14ClN3O,311.77,3.52,3.0,4.0,0.0,3.0,39.94,0.74
4,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL304788,IC50,880.0,Clc1ccc(C(C2Cc3ccccc3O2)n2cncn2)c(Cl)c1,InChI=1S/C17H13Cl2N3O/c18-12-5-6-13(14(19)8-12...,ZSINOYZZEZDTLK-UHFFFAOYSA-N,None,C17H13Cl2N3O,346.22,4.18,3.0,4.0,0.0,3.0,39.94,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448774,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL3329544,Emax,64.0,C[C@H]1C[C@@H]1C(=O)N1CC2CC(C1)N2,InChI=1S/C10H16N2O/c1-6-2-9(6)10(13)12-4-7-3-8...,INYNWEUTASDACG-KRTGUUSXSA-N,None,C10H16N2O,180.25,0.22,0.0,2.0,1.0,1.0,32.34,0.63
1448775,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL3329545,Emax,73.0,CC1(C)CC1C(=O)N1CC2CC(C1)N2,InChI=1S/C11H18N2O/c1-11(2)4-9(11)10(14)13-5-7...,LJEPPQATHWQMBV-UHFFFAOYSA-N,None,C11H18N2O,194.28,0.61,0.0,2.0,1.0,1.0,32.34,0.66
1448776,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL3329546,Emax,110.0,O=C(C1CC1(F)F)N1CC2CC(C1)N2,InChI=1S/C9H12F2N2O/c10-9(11)2-7(9)8(14)13-3-5...,VXQLPXYMYHKASH-UHFFFAOYSA-N,None,C9H12F2N2O,202.20,0.21,0.0,2.0,1.0,1.0,32.34,0.66
1448777,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL4869892,IC50,135.0,CC[C@H](C)[C@@H]1NC(=O)[C@H](CCC(=O)O)NC(=O)[C...,InChI=1S/C68H103N23O21S4/c1-7-32(6)53-66(110)8...,ATDHPNPDBDEQAK-BCTYQQKGSA-N,None,C68H103N23O21S4,1706.98,None,NaN,NaN,NaN,NaN,None,None


In [3]:
df

,Target Name,Target ChEMBL ID,Target Type,Organism,Molecule ChEMBL ID,Activity Type,Activity Value,Canonical SMILES,Standard InChI,Standard InChI Key,Max Phase,Molecular Formula,Molecular Weight,AlogP,Aromatic Rings,HBA,HBD,Rotatable Bonds,Polar Surface Area (PSA),QED Weighted
0,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL63297,IC50,190.0,Fc1ccc(C(C2Cc3ccccc3O2)n2cncn2)cc1,InChI=1S/C17H14FN3O/c18-14-7-5-12(6-8-14)17(21...,RVFPOHIDBASNMF-UHFFFAOYSA-N,None,C17H14FN3O,295.32,3.01,3.0,4.0,0.0,3.0,39.94,0.75
1,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL65328,IC50,590.0,Cc1ccc(C(C2Cc3ccccc3O2)n2cncn2)cc1,InChI=1S/C18H17N3O/c1-13-6-8-14(9-7-13)18(21-1...,OZEGINYEQGNONL-UHFFFAOYSA-N,None,C18H17N3O,291.35,3.18,3.0,4.0,0.0,3.0,39.94,0.74
2,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL488,IC50,18500.0,CCC1(c2ccc(N)cc2)CCC(=O)NC1=O,InChI=1S/C13H16N2O2/c1-2-13(8-7-11(16)15-12(13...,ROBVIMPUHSLWNV-UHFFFAOYSA-N,4.0,C13H16N2O2,232.28,1.35,1.0,3.0,2.0,2.0,72.19,0.60
3,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL291646,IC50,200.0,Clc1ccc(C(C2Cc3ccccc3O2)n2cncn2)cc1,InChI=1S/C17H14ClN3O/c18-14-7-5-12(6-8-14)17(2...,UVOSSULRORMWHE-UHFFFAOYSA-N,None,C17H14ClN3O,311.77,3.52,3.0,4.0,0.0,3.0,39.94,0.74
4,Cytochrome P450 19A1,CHEMBL1978,SINGLE PROTEIN,Homo sapiens,CHEMBL304788,IC50,880.0,Clc1ccc(C(C2Cc3ccccc3O2)n2cncn2)c(Cl)c1,InChI=1S/C17H13Cl2N3O/c18-12-5-6-13(14(19)8-12...,ZSINOYZZEZDTLK-UHFFFAOYSA-N,None,C17H13Cl2N3O,346.22,4.18,3.0,4.0,0.0,3.0,39.94,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448774,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL3329544,Emax,64.0,C[C@H]1C[C@@H]1C(=O)N1CC2CC(C1)N2,InChI=1S/C10H16N2O/c1-6-2-9(6)10(13)12-4-7-3-8...,INYNWEUTASDACG-KRTGUUSXSA-N,None,C10H16N2O,180.25,0.22,0.0,2.0,1.0,1.0,32.34,0.63
1448775,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL3329545,Emax,73.0,CC1(C)CC1C(=O)N1CC2CC(C1)N2,InChI=1S/C11H18N2O/c1-11(2)4-9(11)10(14)13-5-7...,LJEPPQATHWQMBV-UHFFFAOYSA-N,None,C11H18N2O,194.28,0.61,0.0,2.0,1.0,1.0,32.34,0.66
1448776,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL3329546,Emax,110.0,O=C(C1CC1(F)F)N1CC2CC(C1)N2,InChI=1S/C9H12F2N2O/c10-9(11)2-7(9)8(14)13-3-5...,VXQLPXYMYHKASH-UHFFFAOYSA-N,None,C9H12F2N2O,202.20,0.21,0.0,2.0,1.0,1.0,32.34,0.66
1448777,Neuronal acetylcholine receptor; alpha3/alpha6...,CHEMBL2109233,PROTEIN COMPLEX,Homo sapiens,CHEMBL4869892,IC50,135.0,CC[C@H](C)[C@@H]1NC(=O)[C@H](CCC(=O)O)NC(=O)[C...,InChI=1S/C68H103N23O21S4/c1-7-32(6)53-66(110)8...,ATDHPNPDBDEQAK-BCTYQQKGSA-N,None,C68H103N23O21S4,1706.98,None,NaN,NaN,NaN,NaN,None,None


In [4]:
len(list(set(list(df["Target ChEMBL ID"]))))

782

In [5]:
set(list(df["Activity Type"]))

{'% Activity remaining',
 '% Control',
 '% Ctrl',
 '% Enzyme Activity (relative to DMSO controls, average n=2)',
 '% Inhibition of Control Agonist Response (Mean n=2)',
 '% Inhibition of Control Specific Binding',
 '% Inhibition of Control Specific Binding (Mean n=2)',
 '% Inhibition of Control Values',
 '% Inhibition of Control Values (Mean n=2)',
 '% Residual activity with Skepinone-L',
 '% binding',
 '% inhibition',
 '% maximum response',
 '% of Control Agonist Response (Mean n=2)',
 '% of activity',
 '% of control',
 '% of effect',
 '% of engagement',
 '% of inhibition',
 '% of inhibition or stimulation',
 '% of residual activity',
 '% residual kinase activity',
 '%Bound_Albumin',
 '%Inhib (Mean)',
 '%Max (Mean)',
 '%max',
 '-Delta G',
 '-Log ED50',
 '-Log K0.5',
 '-Log KB',
 '-Log KD',
 '-TdeltaS',
 '-log(RatioIC50)',
 '-logKa',
 '1/kmax',
 '1/kobsd-ka',
 'A/Ad',
 'A1 selectivity',
 'A2',
 'A2 selectivity',
 'A50',
 'AC50',
 'ACAT activity',
 'ACT',
 'AE activity',
 'AIDH activity

In [1]:
from transformers import BertTokenizerFast, BertModel
checkpoint = 'unikei/bert-base-smiles'
tokenizer = BertTokenizerFast.from_pretrained(checkpoint)
model = BertModel.from_pretrained(checkpoint)

example = 'O=C([C@@H](c1ccc(cc1)O)N)N[C@@H]1C(=O)N2[C@@H]1SC([C@@H]2C(=O)O)(C)C'
tokens = tokenizer(example, return_tensors='pt')
predictions = model(**tokens)

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/306k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [2]:
predictions

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.3821,  0.0593, -0.1914,  ..., -2.0613, -0.4603, -1.1894],
         [-0.9418, -0.1656,  0.2671,  ..., -0.9770, -0.0633, -1.9062],
         [ 1.1226, -0.4002,  0.3521,  ...,  0.1618, -0.1386,  1.8086],
         ...,
         [ 0.6186,  0.3017,  0.1241,  ..., -0.1342,  0.0974, -0.0602],
         [ 0.4130, -1.1119,  0.2806,  ...,  0.0944, -0.5532, -0.3501],
         [ 0.5375,  0.7749,  1.0559,  ..., -1.0184, -0.0626, -0.3657]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.7910,  0.9982,  0.5162,  0.5645,  0.9931, -1.0000, -0.9679,  0.9552,
         -0.9853,  0.9589, -0.9807,  0.9999,  0.5608, -0.9023,  0.3254,  0.9963,
         -0.9993, -0.9995, -0.9986,  0.9893, -0.9805, -0.9412,  1.0000,  0.9999,
         -0.9650,  0.9021, -0.9906,  0.9998,  0.9825, -0.9002, -0.0313,  0.9987,
         -0.9999,  0.9993, -0.9979, -0.6498, -0.9986,  0.9922, -1.0000,  0.6821,
          0.1280, -0.9983, -0.99